<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2
import plotly.express as px

In [445]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [446]:
# Проигнорируем предупреждения, возникающее при работе с postgresql, для избежания засорения выводов
from warnings import filterwarnings

filterwarnings("ignore", category=UserWarning, message='.*pandas only supports SQLAlchemy connectable.*')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [447]:
# текст запроса
query_3_1 = f'''select count(id)
                from vacancies
'''

In [448]:
# результат запроса
query_3_1_result = pd.read_sql_query(query_3_1, connection)

display(f"Количество вакансий в БД - {query_3_1_result['count'][0]}")

'Количество вакансий в БД - 49197'

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [449]:
# текст запроса
query_3_2 = f'''select count(id)
                from employers
'''

In [450]:
# результат запроса
query_3_2_result = pd.read_sql_query(query_3_2, connection)

display(f"Количество работодателей в БД - {query_3_2_result['count'][0]}")

'Количество работодателей в БД - 23501'

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [451]:
# текст запроса
query_3_3 = f'''select count(id)
                from areas
'''

In [452]:
# результат запроса
query_3_3_result = pd.read_sql_query(query_3_3, connection)

display(f"Количество регионов в БД - {query_3_3_result['count'][0]}")

'Количество регионов в БД - 1362'

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [453]:
# текст запроса
query_3_4 = f'''select count(id)
                from INDUSTRIES
'''

In [454]:
# результат запроса
query_3_4_result = pd.read_sql_query(query_3_4, connection)

display(f"Количество сфер деятельности в БД - {query_3_4_result['count'][0]}")

'Количество сфер деятельности в БД - 294'

***

In [455]:
# выводы по предварительному анализу данных
print(f"Предварительный анализ данных показал, что БД содержит сведения о {query_3_1_result['count'][0]} вакансиях от\n"
        f"{query_3_2_result['count'][0]} работадателя, которые расположены в {query_3_3_result['count'][0]} регионах,\n"
        f"и которые в совокупности предлагают {query_3_4_result['count'][0]} различных сфер деятельности")

Предварительный анализ данных показал, что БД содержит сведения о 49197 вакансиях от
23501 работадателя, которые расположены в 1362 регионах,
и которые в совокупности предлагают 294 различных сфер деятельности


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [456]:
# текст запроса
query_4_1 = f'''select a.name, count(v.id)
                from VACANCIES v left join AREAS a on v.area_id = a.id
                group by a.name
                order by 2 desc
                limit 5
'''

In [457]:
# результат запроса
query_4_1_result = pd.read_sql_query(query_4_1, connection)
query_4_1_result

,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [458]:
# текст запроса
query_4_2 = f'''select count(id)
                from VACANCIES 
                where salary_from is not null or salary_to is not null
'''

In [459]:
# результат запроса
query_4_2_result = pd.read_sql_query(query_4_2, connection)

display(f"У {query_4_2_result['count'][0]} вакансий указан хотя бы один предел по ЗП")

'У 24073 вакансий указан хотя бы один предел по ЗП'

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [460]:
# текст запроса
query_4_3 = f'''select avg(salary_from) salary_from_mean, 
                       avg(salary_to) salary_to_mean
                from VACANCIES
'''

In [462]:
# результат запроса
query_4_3_result = pd.read_sql_query(query_4_3, connection)
print(f"Средняя нижняя граница зарплатной вилки {query_4_3_result['salary_from_mean'][0]:.0f}\n"
        f"Средняя верхняя граница зарплатной вилки {query_4_3_result['salary_to_mean'][0]:.0f}")

Средняя нижняя граница зарплатной вилки 71065
Средняя верхняя граница зарплатной вилки 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [463]:
# текст запроса
query_4_4 = f'''select schedule, employment, count(id)
                from VACANCIES
                group by 1, 2
                order by 3 desc
'''

In [464]:
# результат запроса
query_4_4_result = pd.read_sql_query(query_4_4, connection)
query_4_4_result

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [465]:
# текст запроса
query_4_5 = f'''select experience, count(id)
                from VACANCIES
                group by 1
                order by 2
'''

In [466]:
# результат запроса
query_4_5_result = pd.read_sql_query(query_4_5, connection)
query_4_5_result

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [359]:
# выводы по детальному анализу вакансий
# Первые пять мест при группировке по регионам занимают Москва, Санкт-Петербург, Минск, Новосибирск и Алматы
# Примерно в 50 % случаев не указаны пределы по ЗП
# 72 % всех вакансий имеют формат занятости - полная занятость, а график - полный день 
# Средняя нижняя граница зарплатной вилки - 71065, средняя верхняя граница зарплатной вилки - 110537
# 50 % всех вакансий требуют опыт работы от 1 года до 3 лет

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [467]:
# текст запроса
query_5_1 = f'''select e.name, count(v.id)
                from VACANCIES v left join EMPLOYERS e on v.employer_id = e.id
                group by 1
                order by 2 desc
                limit 5
'''

In [468]:
# результат запроса
query_5_1_result = pd.read_sql_query(query_5_1, connection)
query_5_1_result

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [469]:
# текст запроса
query_5_2 = f'''select a.name,
                       count(e.id) employ_count,
                       count(v.id) vac_count
            from areas a
                left join employers e on a.id = e.area
                left join vacancies v on v.area_id = a.id
            group by 1
            order by 3 asc, 2 desc
'''

In [470]:
# результат запроса
query_5_2_result = pd.read_sql_query(query_5_2, connection)
query_5_2_result

,name,employ_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Новосибирск,1149438,1149438
1358,Алматы,1364132,1364132
1359,Минск,2354880,2354880
1360,Санкт-Петербург,6320667,6320667


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [471]:
# текст запроса
query_5_3 = f'''select e.name,
                       count(distinct v.area_id)
                from EMPLOYERS e 
                left join vacancies v on e.id = v.employer_id
                group by e.id
                order by 2 desc
'''

In [472]:
# результат запроса
query_5_3_result = pd.read_sql_query(query_5_3, connection)
query_5_3_result

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [473]:
# текст запроса
query_5_4 = f'''select count(*)
                from EMPLOYERS e left join EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id 
                where ei.industry_id is null
'''

In [474]:
# результат запроса
query_5_4_result = pd.read_sql_query(query_5_4, connection)
print(f"Количество работодателей, у которых не указана сфера деятельности - {query_5_4_result['count'][0]}")

Количество работодателей, у которых не указана сфера деятельности - 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [475]:
# текст запроса
query_5_5 = f'''select e.name,
                       count(industry_id)
                from EMPLOYERS e left join EMPLOYERS_INDUSTRIES ei on e.id = ei.employer_id
                group by 1 
                having count(industry_id) = 4
                order by e.name
                offset 2
                limit 1
'''

In [476]:
# результат запроса
query_5_5_result = pd.read_sql_query(query_5_5, connection)
print(f"Название компании, которая находится на третьем месте по алфавиту среди компаний с 4-мя сферами деятельности - {query_5_5_result['name'][0]}")

Название компании, которая находится на третьем месте по алфавиту среди компаний с 4-мя сферами деятельности - 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [477]:
# текст запроса
query_5_6 = f'''select count(*)
                from EMPLOYERS_INDUSTRIES ei 
                left join INDUSTRIES i on ei.industry_id = i.id
                where i.name like 'Разработка программного обеспечения'
'''

In [478]:
# результат запроса
query_5_6_result = pd.read_sql_query(query_5_6, connection)
print(f"Разработкой ПО занимаются {query_5_6_result['count'][0]} компании")

Разработкой ПО занимаются 3553 компании


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [479]:
# код для получения списка городов-милионников
# Импортируем библиотеку requests
import requests 
# Импортируем библиотеку BeautifulSoup
from bs4 import BeautifulSoup 

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

table = page.find('table', class_='standard sortable').find_all('tr') # Находим таблицу с городами-миллионниками 

million_city = [] # Инициализируем список, куда будут добавлены города

for elem in table[1:]: # Запускаем цикл по элементам поиска по веб-странице
    million_city.append(elem.find('a').text) # Заполняем список 

million_city = tuple(million_city) # Преобразуем список в кортеж, чтобы воспользоваться в запросе
print(million_city)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [480]:
# текст запроса
query_5_7 = f'''(select a.name,
                       count(v.id)
                from vacancies v 
                left join EMPLOYERS e on v.employer_id = e.id
                left join AREAS a on v.area_id = a.id
                where e.name like 'Яндекс' and a.name in {million_city}
                group by 1
                order by 2)
                
                union all
                
                (select 'total',
                        count(*) total
                from vacancies v 
                left join EMPLOYERS e on v.employer_id = e.id
                left join AREAS a on v.area_id = a.id
                where e.name like 'Яндекс' and a.name in {million_city}
                group by 1)
'''

In [481]:
# результат запроса
query_5_7_result = pd.read_sql_query(query_5_7, connection)
query_5_7_result


,name,count
0,total,485
1,Омск,21
2,Челябинск,22
3,Красноярск,23
4,Волгоград,24
5,Пермь,25
6,Казань,25
7,Ростов-на-Дону,25
8,Уфа,26
9,Самара,26


***

In [375]:
# выводы по анализу работодателей
# Компания "Яндекс" предлагает почти в 4 раза больше вакансий, а именно 1993, чем следующая за ней компания "Ростелеком", 485 из которых предлагаются в городах-миллионниках
# Соподсчет работодателей и вакансий по регионам показал, что некоторые регионы отображают только принадлежность компании определенному местоположению, 
# к вакансиям такие регионы отношения не имеют
# "Яндекс" также является лидером количеству регионов, в котором предлагаются вакансии компании
# По результатам запроса устновлено, что 36 % работодателей не указывают сферу деятельности
# Разработкой ПО занимаются 3553 компании

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [482]:
# текст запроса
query_6_1 = f'''select count(v.id)
                from vacancies v
                where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
'''

In [483]:
# результат запроса
query_6_1_result = pd.read_sql_query(query_6_1, connection)
print(f"{query_6_1_result['count'][0]} вакансия имеет отношение к данным")

1771 вакансия имеет отношение к данным


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [484]:
# текст запроса
query_6_2 = f'''select count(v.id)
                from vacancies v
                where (lower(v.name) like '%data scientist%'
                      or lower(v.name) like '%data science%'
                      or lower(v.name) like '%исследователь данных%'
                      or lower(v.name) like '%машинн%обучен%'
                      or lower(v.name) like '%machine learning%'
                      or (v.name like '%ML%' and v.name not like '%HTML%')) 
                      
                      and (lower(name) like '%junior%'
                          or experience = 'Нет опыта'
                          or employment = 'Стажировка')
'''

In [485]:
# результат запроса
query_6_2_result = pd.read_sql_query(query_6_2, connection)
print(f"{query_6_2_result['count'][0]} вакансия подходит для начинающего дата-сайентиста")

51 вакансия подходит для начинающего дата-сайентиста


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [486]:
# текст запроса
query_6_3 = f'''select count(v.id)
                from vacancies v
                where (lower(v.name) like '%data scientist%'
                      or lower(v.name) like '%data science%'
                      or lower(v.name) like '%исследователь данных%'
                      or lower(v.name) like '%машинн%обучен%'
                      or lower(v.name) like '%machine learning%'
                      or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')) 
                      
                      and (lower(v.key_skills) like '%sql%'
                          or lower(v.key_skills) like '%postgres%')
'''

In [487]:
# результат запроса
query_6_3_result = pd.read_sql_query(query_6_3, connection)
print(f"{query_6_3_result['count'][0]} вакансий требуют знание SQL и Postrges")

229 вакансий требуют знание SQL и Postrges


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [488]:
# текст запроса
query_6_4 = f'''select count(v.id)
                from vacancies v
                where (lower(v.name) like '%data scientist%'
                      or lower(v.name) like '%data science%'
                      or lower(v.name) like '%исследователь данных%'
                      or lower(v.name) like '%машинн%обучен%'
                      or lower(v.name) like '%machine learning%'
                      or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')) 
                      
                      and (lower(v.key_skills) like '%python%')
'''

In [489]:
# результат запроса
query_6_4_result = pd.read_sql_query(query_6_4, connection)
print(f"{query_6_4_result['count'][0]} вакансий требуют знание Python")

357 вакансий требуют знание Python


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [490]:
# текст запроса
query_6_5 = f'''select v.key_skills 
                from vacancies v
                where v.key_skills is not null 
                      and (lower(v.name) like '%data scientist%'
                          or lower(v.name) like '%data science%'
                          or lower(v.name) like '%исследователь данных%'
                          or lower(v.name) like '%машинн%обучен%'
                          or lower(v.name) like '%machine learning%'
                          or (v.name like '%ML%' and lower(v.name) not like '%html%'))
'''

In [491]:
# результат запроса
query_6_5_result = pd.read_sql_query(query_6_5, connection)

# Напишем фeнкцию для преобразования признака
def mean_key_skills(arg):
    '''Функция преобразует признак ключевые навыки в признак среднего числа ключевых навыков

    На выходе из функции остается число, указанных навыков

    Args:
        arg (srting): перечисление ключевых навыков

    Returns:
        int: число, указанных навыков
    '''
    # Развиваем строку по разделителю
    arg = arg.split('\t')
    # Возвращаем длину списка
    return len(arg)

# Применяем функцию mean_key_skills к столбцу из полученного запроса
query_6_5_result['mean_key_skills'] = query_6_5_result['key_skills'].apply(mean_key_skills)
print(f"{query_6_5_result['mean_key_skills'].mean():.2f} ключевых навыков в среднем необходимо для DS")

6.41 ключевых навыков в среднем необходимо для DS


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [492]:
# текст запроса
query_6_6 = f'''select experience,
                       round(avg(
                          coalesce((salary_from + salary_to)/2, salary_from, salary_to)
                ), 0)
                from vacancies v
                where (salary_from is not null or salary_to is not null)
                      and (lower(v.name) like '%data scientist%'
                          or lower(v.name) like '%data science%'
                          or lower(v.name) like '%исследователь данных%'
                          or lower(v.name) like '%машинн%обучен%'
                          or lower(v.name) like '%machine learning%'
                          or (v.name like '%ML%' and lower(v.name) not like '%html%'))
                group by 1
'''

In [493]:
# результат запроса
query_6_6_result = pd.read_sql_query(query_6_6, connection)
query_6_6_result

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [441]:
# выводы по предметному анализу
# 1771 вакансия имеет отношение к данны
# 51 вакансия подходит для начинающего дата-сайентиста
# 229 вакансий требуют знание SQL и Postrges
# 357 вакансий требуют знание Python
# 6.41 ключевых навыков в среднем необходимо для DS

# Общий вывод по проекту

In [389]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

**Выводы по предварительному анализу данных:**

Предварительный анализ данных показал, что БД содержит сведения о 49197 вакансиях от 23501 работадателя, которые расположены в 1362 регионах, и которые в совокупности предлагают 294 различных сфер деятельности 

**Для обобщения выводов по анализу вакансий визуализируем некоторые данные, для этого:** 


In [494]:
# Посчитаем сколько процентов всех предлагаемых вакансий приходятся на 5 самых популярных городов, ответ округлим до целого значения
areas_percent_pop = query_4_1_result['count'].sum()/query_3_1_result['count'][0] * 100
print(f'Процент вакансий, которые приходятся на 5 самых популярных региона, составляет {areas_percent_pop:.0f}%')

# Найдем сколько при этом процентов вакансий приходится на Москву
mosc_percent_vac = query_4_1_result['count'][0]/query_4_1_result['count'].sum() * 100
print(f'Среди самых популярных регионов Москва занимает {mosc_percent_vac:.0f}% предлагаемых вакансий')

# Дополним таблицу с числом вакансий по регионам суммарым числом остальных регионов, обозначим "другие"
areas_distrib = query_4_1_result.copy()
areas_distrib.loc[5] = ['Другие', query_3_1_result['count'][0] - query_4_1_result['count'].sum()]

# Построим круговую диаграмму для отображения распределения вакансий по регионам
areas_distrib = px.pie(
    data_frame=areas_distrib,
    names='name',
    values ='count',
    title='Рисунок 1 - Процентное распредление вакансий по регионам'
)

areas_distrib.write_html('Визуализация/Fig1-Рисунок_1_Распредление_вакансий_по_регионам.html')

Процент вакансий, которые приходятся на 5 самых популярных региона, составляет 29%
Среди самых популярных регионов Москва занимает 38% предлагаемых вакансий


In [495]:
# Построим круговую диаграмму для визуализации процентного распределения вакансий по требуемому опыту работы
exp_distrib = px.pie(
    data_frame=query_4_5_result,
    names='experience',
    values ='count',
    title='Рисунок 2 - Процентное распредление вакансий по требуемому опыту работы'
)

exp_distrib.write_html('Визуализация/Fig2-Рисунок_2_Распредление_вакансий_по_ОР.html')

In [ ]:
# Создадим дополнительный запрос для исследования распределения ЗП
query_4_6 = f'''select salary_from "Нижняя граница зарплатной вилки",
                       salary_to "Верхняя граница зарплатной вилки"
                from vacancies v
'''

# Запищем результат запроса в датафрейм
query_4_6_result = pd.read_sql_query(query_4_6, connection)

# Построим коробчатую диаграмму для анализа распределения
salary_dist = px.box(
    data_frame=query_4_6_result,
    title='Рисунок 3 - Распределение нижней и верхней границ зарплатной вилки'
)

 # Добавим подписи к осям и сохраним в html формат
salary_dist.update_layout(xaxis_title="", yaxis_title="ЗП, руб")
salary_dist.write_html('Визуализация/Fig3-Распределение_нижней_и_верхней_границ_зарплатной_вилки.html')

In [496]:
# Создадим запрос для построения столбчатой диаграммы для компаний , предлагающих более 100 вакансий в различных регионах
query_5_1 = f'''select e.name, count(v.id)
                from VACANCIES v left join EMPLOYERS e on v.employer_id = e.id
                group by 1
                having count(v.id) > 100
                order by 2 desc
'''

# Создадим запрос для построения столбчатой диаграммы для компаний, предлагающих вакансии в более 50 регионах
query_5_3 = f'''select e.name,
                       count(distinct v.area_id)
                from EMPLOYERS e 
                left join vacancies v on e.id = v.employer_id
                group by e.id
                having count(distinct v.area_id) > 50
                order by 2 desc
'''

# Создадим запрос для построения столбчатой диаграммы для компании "Яндекс" по городам-миллионникам
query_5_7 = f'''(select a.name,
                       count(v.id)
                from vacancies v 
                left join EMPLOYERS e on v.employer_id = e.id
                left join AREAS a on v.area_id = a.id
                where e.name like 'Яндекс' and a.name in {million_city}
                group by 1
                order by 2)
'''

# Сохраним результат запроса в датафрейм
query_5_1_result = pd.read_sql_query(query_5_1, connection)
query_5_3_result = pd.read_sql_query(query_5_3, connection)
query_5_7_result = pd.read_sql_query(query_5_7, connection)

# Построим столбчатую диаграмму 
employ_bar = px.bar(
    data_frame=query_5_1_result,
    x='name',
    y='count',
    title= 'Рисунок 4 - Компании, которые предлагают более 100 вакансий'
)
area_bar = px.bar(
    data_frame=query_5_3_result,
    x='name',
    y='count',
    title= 'Рисунок 5 - Компании, которые предлагают вакансии в более чем 50 регионах'
)
yandex_bar = px.bar(
    data_frame=query_5_7_result,
    x='name',
    y='count',
    title= 'Рисунок 6 - Распределение вакансий, предлагаемых команией "Яндекс" в городах-миллионниках'
)

 # Добавим подписи к осям и сохраним в html формат
employ_bar.update_layout(xaxis_title="", yaxis_title="Колличество вакансий, ед.")
employ_bar.write_html('Визуализация/Fig4-Комп_где_более_100_вакансий.html')

area_bar.update_layout(xaxis_title="", yaxis_title="Колличество вакансий, ед.")
area_bar.write_html('Визуализация/Fig5-Комп_где_более_50_регионов.html')

yandex_bar.update_layout(xaxis_title="", yaxis_title="Колличество вакансий, ед.")
yandex_bar.write_html('Визуализация/Fig6-yandex.html')

In [497]:
# Посчитаем процент работодалетей, которые не указывают сферы деятельности компаний 
field_percent = query_5_4_result['count'][0]/query_3_2_result['count'][0] * 100
print(f"{field_percent:.0f} % работодателей не указывают сферу деятельности")

36 % работодателей не указывают сферу деятельности


In [498]:
# Построим дополнительную даиграмму для визуализации востребованности вакансий 
field_distib = f'''select v.name,
                          count(v.id)
                   from vacancies v
                   group by 1
                   having count(v.id) > 100
                   order by 2
'''

# Сохраним результат запроса в датафрейм
field_distib_result = pd.read_sql_query(field_distib, connection)

# Построим горизонтальную столбчатую диаграмму 
field_distib_bar = px.bar(
    data_frame=field_distib_result,
    x='count',
    y='name',
    color='count',
    title= 'Рисунок 8 - Наиболее востребованные вакансии'
)

 # Добавим подписи к осям и сохраним в html формат
field_distib_bar.update_layout(xaxis_title="Количествоб, ед", yaxis_title="Вакансия")
field_distib_bar.write_html('Визуализация/Fig8-Востребованность вакансий.html')

In [499]:
# Вычислим процент вакансий, относящихся к данным
vac_data_percent = query_6_1_result['count'][0]/query_3_1_result['count'][0] *100
print(f"{vac_data_percent:.0f} % вакансий относятся к сфере работы с данными")

4 % вакансий относятся к сфере работы с данными


In [500]:
# Дополнительно найдем общее количество вакансий из области DS
query_6_7 = f'''select count(v.id)
                from vacancies v
                where (lower(v.name) like '%data scientist%'
                      or lower(v.name) like '%data science%'
                      or lower(v.name) like '%исследователь данных%'
                      or lower(v.name) like '%машинн%обучен%'
                      or lower(v.name) like '%machine learning%'
                      or (v.name ilike '%ML%' and v.name not ilike '%HTML%'))
'''
# Запишем результат в переменную
query_6_7_result = pd.read_sql_query(query_6_7, connection)

# Найдем процент вакансий, где требуется знание SQL, Postgres
sql_persent = query_6_3_result['count'][0]/query_6_7_result['count'][0] * 100
print(f"{sql_persent:.0f} % вакансий требуют знания SQL, Postgres")

# Найдем процент вакаансий, где требуется знание Python
sql_persent = query_6_4_result['count'][0]/query_6_7_result['count'][0] * 100
print(f"{sql_persent:.0f} % вакансий требуют знания Python")

# Построим столбчатую диаграмму 
exp_sal_bar = px.bar(
    data_frame=query_6_6_result,
    x='experience',
    y='round',
    color='experience',
    title= 'Рисунок 7 - Средняя зарплата в зависимости от опыта работы в сфере DS'
)

 # Добавим подписи к осям и сохраним в html формат
exp_sal_bar.update_layout(xaxis_title="", yaxis_title="ЗП, руб.")
exp_sal_bar.write_html('Визуализация/Fig7-ЗП от ОР.html')

43 % вакансий требуют знания SQL, Postgres
67 % вакансий требуют знания Python


**Выводы по детальному анализу вакансий:**

1. <u>29 %</u> всех вакансий приходятся на <u>Москву, Санкт-Петербург, Минск, Новосибирск и Алматы</u>, при этом 38 % из них приходятся на Москву. Вероятно такое распределение связано с тем, что большинство крупных компаний предлагающих большое колличество вакансий расположено в Москве, что в свою очередь может быть результатом привлектельности Москвы как города для проживания, а также повышенные зарплатные ожидания. Остальные города также являются центрами своих областей и даже столицей, которые отвечают повышенному уровню жизни и ведения бизнеса. 

<center> <img src=Fig1.png width=1000 height=550> </center>

2. 53 % всех вакансий требуют опыта работы от 1 года до 3 лет. Опыт работы является ключевым фактором, влияющим на заработную плату при приеме на работу. Вероятно такое распределение получилось в результате того, что именно под эту категорию попадает большее количество кандидатов. Также можно предположить, что опытные специлисте реже увольняются и переходят на другую работу.

<center> <img src=Fig2.png width=1000 height=550> </center>

3. Средняя нижняя граница зарплатной вилки - 71065, средняя верхняя граница зарплатной вилки - 110537. По распредилению нижней и верхней границ зарплатной вилки видно, что распределения нормальными не являются, так как сильно смещенны в большую сторону, имеют большое колличество возможных выбросов,которые могут и не быть выбросами так, как ЗП зависит от многих факторов таких, как регион компании и сама компания, сфреа деятельности компании, а медианные и средние значения при этом сильно отличаются. Также по результатам запросов установленно, что около 50% работадателей не указывают ни один из пределов, что делает анализ распределений менее точным из-за малости выборки.

<center> <img src=Fig3.png width=1000 height=550> </center>

4. Среди работадателей "Яндекс" является явным лидером, эта компания предлагает 1933 выкансии в 181 регионе (рисунки 4 и 5 соответсвенно), компания "Яндекс" является одной из крупнейших в России IT компаний, которые постоянно расширяются, поэтому такое количество вакансий не удивительно, также IT профессия часто связана с удаленной работой, в следствии чего компания может предлагать работу в незасимости от места проживания.

<center> <img src=Fig4.png width=1000 height=550> </center>
<center> <img src=Fig5.png width=1000 height=550> </center>

5. На рисунке 6 представлено распределение вакансий, предлагаемых компанией "Яндекс" в городах-миллионниках, наибольшее количество вакансий предлогается в Москве и Санкт-Петербурге

<center> <img src=Fig6.png width=1000 height=550> </center>

6. По результатам запросов установлено, что 43% вакансий, связанных с DS, требуют знаний SQL, Postgres, а знания языка программирования Python требуется в 67% случаев, на основе полученных данных можно сказать, что SQL и Python являются ключевыми инструментыми в DS, однако это не весь список ключевых навыков, результаты запроса также показывают, что для устройства по вакансии DS необходимо в среднем обладать минимум 6 ключевыми навыками

7. На рисунке 7 показано распределение средней зарплаты дата-сайентиста в зависимости от опыта работы, на рисунке 7 можно увидеть закономерное увеличение ЗП с увеличением опыта работы

<center> <img src=fig7.png width=1000 height=550> </center>

8. Среди размещенных предложений о работе вакансии IT специалистов занимают лидирующие позиции. Это связано со всеобщим расширением данной области и нехватки квалифицированных кадров

<center> <img src=Fig8.png width=1000 height=550> </center>

**Возможные дополнительные исследования:**

1. Интересным полученным результатом запроса является то, что примерно в 50 % случаев не указаны пределы по ЗП, причины и закономерности этого могли бы стать предметом отдельного исследования

2. 72 % всех вакансий имеют формат занятости - полная занятость, а график - полный день, влияние данного фактора на размер ЗП также может быть предметом исследования



In [501]:
connection.close()